# Apache Spark Fundamentals: RDDs


In this notebook we will work with the RDDs that are part of the Spark Core. The implementation of Spark Core is a **RDD (Resilient Distributed Dataset)** which is a collection of data distributed in different nodes of the cluster that are processed in parallel.

We will use the PySpark API, but the concepts apply equally to all APIs (Scala, R, etc)

### Initializing Spark on Notebooks

In [1]:
#!conda install -c conda-forge findspark

In [2]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [3]:
from pyspark.sql import SparkSession

### Create the SparkSession and SparkContext

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_training')\
        .getOrCreate()

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Create an RDD from a collection

In [6]:
num = [1,2,3,4,5]

num_rdd = sc.parallelize(num)
num_rdd.collect()

[1, 2, 3, 4, 5]

# Transformations
* Transformations are lazy in nature and will not be executed until an Action is executed on them.
* Let's try to understand the different transformations available.

### map
* This will map your input to some output based on the function specified in the function

In [7]:
double_rdd = num_rdd.map(lambda x : x * 2)
double_rdd.collect()

[2, 4, 6, 8, 10]

### filter
* To filter the data based on a certain condition. Let's try to find the even numbers of num_rdd.

In [8]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0)
even_rdd.collect()

[2, 4]

### flatMap
* This function is very similar to map, but it can return multiple elements for each entry in the given RDD.

In [9]:
flat_rdd = num_rdd.flatMap(lambda x : range(1,x))
flat_rdd.collect()

[1, 1, 2, 1, 2, 3, 1, 2, 3, 4]

### distinct
* This will return items other than an RDD.

In [10]:
rdd1 = sc.parallelize([10, 11, 10, 11, 12, 11])
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[10, 11, 12]

### reduceByKey
* This function reduces key value pairs based on keys and a given function within reduceByKey

In [11]:
pairs = [ ("a", 5), ("b", 7), ("c", 2), ("a", 3), ("b", 1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y : x + y)

result = output.collect()
print(*result, sep='\n')

('a', 8)
('b', 8)
('c', 6)


### groupByKey
* This function is another ByKey function that can operate on an RDD (key, value pair)  but this will only group the values based on the keys. In other words, this will only perform the first step of reduceByKey.

In [12]:
grp_out = pair_rdd.groupByKey()
grp_out.collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x1a7deafc4f0>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x1a7deafc5b0>),
 ('c', <pyspark.resultiterable.ResultIterable at 0x1a7deafc730>)]

### sortByKey
* This function will perform sorting on an RDD (key, value) pair based on the keys. By default, the sorting will be done in ascending order.

In [13]:
pairs = [ ("a", 5), ("d", 7), ("c", 2), ("b", 3)]
raw_rdd = sc.parallelize(pairs)

sortkey_rdd = raw_rdd.sortByKey()
result = sortkey_rdd.collect()
print(*result,sep='\n')

# Para clasificar en orden descendente, pase  “ascending=False”.

('a', 5)
('b', 3)
('c', 2)
('d', 7)


### Sort by
* sortBy is a more general function for sorting.

In [14]:
# Create RDD.
pairs = [ ("a", 5, 10), ("d", 7, 12), ("c", 2, 11), ("b", 3, 9)]
raw_rdd = sc.parallelize(pairs)

# Let’s try to do the sorting based on the 3rd element of the tuple.
sort_out = raw_rdd.sortBy(lambda x : x[2])
result = sort_out.collect()
print(*result, sep='\n')

('b', 3, 9)
('a', 5, 10)
('c', 2, 11)
('d', 7, 12)


# Actions

* Actions are operations on RDD that are executed immediately. While transformations return another RDD, actions return native data structures

### count
* This will count the number of items in the given RDD.

In [15]:
num = sc.parallelize([1,2,3,4,2])
num.count()

5

### first
* This will return the first element of the given RDD.

In [16]:
num.first()

1

### Collect
* This will return all elements for the given RDD.


In [17]:
num.collect()

[1, 2, 3, 4, 2]

**We should not use the collect operation while working with large data sets**. Because it will return all the data that is distributed between the different workers of the cluster to a controller. All the data will travel through the network from the worker to the driver and also the driver would need to store all the data. This will hamper the performance of your application.

### Take
* This will return the number of items specified.

In [18]:
num.take(3)

[1, 2, 3]